# TP4 - Exercício 1
## Grupo 1

*   Diogo Coelho da Silva A100092
*   Pedro Miguel Ramôa Oliveira A97686


#### 1. Import's

In [44]:
from z3 import *

#### 2.Definição do algoritmo de euclides



In [45]:
from z3 import *

def algoritmoEuclides(num1, num2):
    r_prev, r_curr = num1, num2
    s_prev, s_curr = 1, 0
    t_prev, t_curr = 0, 1
    while r_curr != 0:
        q = r_prev // r_curr
        r_prev, r_curr = r_curr, r_prev - q * r_curr
        s_prev, s_curr = s_curr, s_prev - q * s_curr
        t_prev, t_curr = t_curr, t_prev - q * t_curr
    return r_prev

# Example
print(algoritmoEuclides(12, 18))

6


In [46]:
from z3 import *

def Prova(formula):
    """
    Esta função tenta provar uma fórmula lógica por refutação.
    Basicamente, adiciona a negação da fórmula a um solver SAT (Satisfiability)
    e verifica se o resultado é satisfazível.

    Se a negação da fórmula for insatisfazível, significa que a fórmula original
    é uma tautologia (sempre verdadeira), ou seja, provada.

    """

    # Cria uma instância de um solver Z3. 
    s = Solver()
    # Adiciona a negação da fórmula ao solver. A ideia é mostrar que a negação leva a uma contradição, provando assim a fórmula original.
    s.add(Not(formula))
    # Verifica se o conjunto de restrições no solver (neste caso, apenas a negação da fórmula) é satisfazível. Se for satisfazível, significa que existe uma atribuição de valores
    # que torna a negação verdadeira, e portanto a fórmula original falsa em algumas situações.
    if s.check() == sat:
        print("Falha ao provar a propriedade.")
    else:
        print("Propriedade provada com sucesso.")

# Declara um conjunto de variáveis inteiras que serão usadas para representar os estados do algoritmo e os seus valores.
x, y, r, rr, s, ss, t, tt, q = Ints('x y r rr s ss t tt q')

# Define uma função recursiva para calcular o máximo divisor comum (MDC) de dois números inteiros. `RecFunction` é usado para definir funções recursivas dentro do contexto do Z3.
gcd_func = RecFunction('gcd_func', IntSort(), IntSort(), IntSort())
# Declara duas variáveis inteiras `a` e `b` que serão usadas como argumentos para a função `gcd_func`.
a, b = Ints('a b')
# Neste caso, usa a definição padrão do algoritmo de Euclides para o MDC:
# Se `b` for 0, o MDC é `a`. Caso contrário, o MDC de `a` e `b` é o mesmo que o MDC de `b` e o resto da divisão de `a` por `b` (`a % b`).
RecAddDefinition(gcd_func, (a, b), If(b == 0, a, gcd_func(b, a % b)))

# Define a pré-condição do algoritmo. Neste caso, exige que os valores iniciais de `x` e `y` sejam positivos.
pre_con = And(x > 0, y > 0)

# Define o estado inicial do algoritmo. Aqui, `r` e `rr` recebem os valores iniciais de `x` e `y`, e as variáveis `s`, `ss`, `t`, `tt` são inicializadas para acompanhar os coeficientes da identidade de Bézout.
init_state = And(r == x, rr == y, s == 1, ss == 0, t == 0, tt == 1)

# Define o invariante de loop. Um invariante de loop é uma condição que é verdadeira antes, durante e depois de cada iteração de um loop.
# Aqui, o invariante garante duas coisas:
# 1. O MDC de `x` e `y` é igual ao MDC de `r` e `rr`. Isso significa que o MDC não muda ao longo do algoritmo.
# 2. As igualdades de Bézout são mantidas: `r` pode ser expresso como uma combinação linear de `x` e `y` (`r == x*s + y*t`),
#    e `rr` também pode ser expresso como uma combinação linear de `x` e `y` (`rr == x*ss + y*tt`).
loop_inv = And(gcd_func(x, y) == gcd_func(r, rr),
               r == x*s + y*t,
               rr == x*ss + y*tt)

# Define a condição pré-loop. Esta afirma que se a pré-condição (`pre_con`) e o estado inicial (`init_state`) forem verdadeiros,
# então o invariante de loop (`loop_inv`) também será verdadeiro no início do loop.
pre_loop = And(pre_con, init_state)
# `Implies` cria uma implicação lógica. `pre_final` afirma que `pre_loop` implica `loop_inv`.
# Isto é, se a pré-condição e o estado inicial são satisfeitos, então o invariante do loop também é satisfeito no início.
pre_final = Implies(pre_loop, loop_inv)

# Define a condição de saída do loop. O loop termina quando `rr` for igual a 0.
# Além disso, o invariante de loop (`loop_inv`) deve continuar válido quando o loop termina.
# `ForAll` quantifica universalmente as variáveis `r`, `rr`, `s`, `ss`, `t`, `tt`, significando que a condição deve ser verdadeira para todos os valores dessas variáveis.
exit_cond = ForAll([r, rr, s, ss, t, tt], And(rr == 0, loop_inv))

# Define a pós-condição do algoritmo. Esta descreve o estado desejado após a execução do algoritmo.
# Aqui, a pós-condição afirma que:
# 1. O valor final de `r` é igual ao MDC de `x` e `y`.
# 2. A igualdade de Bézout é satisfeita com os valores finais de `r`, `s` e `t`.
post_con = And(r == gcd_func(x, y),
               r == x*s + y*t)
# `post_final` afirma que se a condição de saída (`exit_cond`) for verdadeira, então a pós-condição (`post_con`) também será verdadeira.
# Isto é, se o loop termina corretamente, então o resultado obtido (o valor de `r`) é o MDC e a identidade de Bézout é mantida.
post_final = Implies(exit_cond, post_con)

# Prova a especificação do algoritmo. A especificação é que se a pré-condição é atendida,
# então a pós-condição será satisfeita quando o algoritmo terminar (se terminar).
# A prova é feita verificando se a afirmação que combina `pre_final` e `post_final`
# (após ser implicada por `True`, o que não altera o seu valor de verdade) é insatisfatível.
# Se for insatisfatível, significa que não há nenhuma atribuição de valores que torne a negação dessa afirmação verdadeira,
# o que implica que a afirmação original é sempre verdadeira, provando a correção do algoritmo.
Prova(Implies(True, And(pre_final, post_final)))

Propriedade provada com sucesso.
